In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
df_item['type'].value_counts()

In [ ]:
df_item[df_item['type']=='SingleQuestion'].groupby(['variable_name'])['value'].value_counts()

In [ ]:
filter_condition = (self.df_item['type']=='SingleQuestion')
pivot_table, index_col = self.get_clean_pivot_table('value',remove_low_freq_col=True, filter_conditions=filter_condition)

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns[0:12]:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    entropy_ = data.groupby('responsible')[col].apply(calculate_entropy)
    #entropy_ = entropy_.reset_index()
    #data[col+'_entropy'] = data['responsible'].map(entropy_[col])
    entropy_.plot(kind='bar')
    plt.title(col)
    plt.show()

In [ ]:
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table.copy()#[~pd.isnull(pivot_table[col])].copy()
    entropy_ = data.groupby('responsible')[col].apply(calculate_entropy)
    entropy_ = entropy_.reset_index()
    pivot_table[col+'_entropy'] = pivot_table['responsible'].map(entropy_.set_index('responsible')[col])

In [ ]:
selected_columns = pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns
selected_columns = [col for col in selected_columns if 'answer_set_anomaly' in col]
pivot_table['total_selected_columns'] = pivot_table[selected_columns].sum(1)

In [ ]:
entropy_columns = [col for col in pivot_table.columns if 'entropy' in col]
pivot_table['entropy__single_question'] = pivot_table[entropy_columns].fillna(-1).mean(1)
entropy_ = pivot_table.groupby('responsible')['entropy__single_question'].mean()
entropy_.plot(kind='bar')
plt.title('entropy__single_question')
plt.show()